In [3]:
import sqlite3
import pandas as pd;
import os
#permet de récuperer une liste de fichiers 
import glob 

In [4]:
#parcourir le dossier avec les csv / Remplace 'chemin_vers_vos_csv/*.csv'
list_csvs = glob.glob('./csv/*.csv')
print(list_csvs) 


[]


In [5]:
#Bdd temporaire stockée en mémoire , connexion 
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [6]:
#Lire chaque fichier CSV et l'insérer en tant que table dans SQLite

for one_csv in list_csvs:
     # Lire le CSV dans un DataFrame pandas
    df = pd.read_csv(one_csv)
   
   # Utiliser os.path.basename pour obtenir le nom du fichier avec extension
    file_name = os.path.basename(one_csv) # Récupère le nom de fichier avec extension

    print(df.head())
    print(file_name)

    # Divise le nom du fichier en une liste de sous - chaines qui utilise le caractère '.'
    # ensuite [0] séléctionne le premier élément de la liste 
    table_name = file_name.split('.')[0]


    #inserer le DF dans la table sqlite temporare / la base de donnée en mémoire créer plus haut 

    df.to_sql(table_name, conn, index=False, if_exists='replace')
    
    print(f"Table '{table_name}' créée avec succès dans SQLite.\n")

In [7]:
# 4. Vérifier les tables créées dans la base de données SQLite
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables créées dans la base de données SQLite :")
print(tables)
print("Tables disponibles dans la base de données :")
for table in tables:
    print(table[0])

Tables créées dans la base de données SQLite :
[]
Tables disponibles dans la base de données :


In [8]:
if tables:  # Vérifie s'il y a des tables créées
    table_name = tables[0][0]  # Prendre le nom de la première table
    request = f"SELECT * FROM {table_name} LIMIT 5;"  # Requête SQL
    resultats = pd.read_sql_query(request, conn)

    # 6. Afficher les résultats de la requête
    print(f"\nRésultats de la requête sur la table '{table_name}':")
    print(resultats)
else:
    print("Aucune table n'a été créée dans la base de données SQLite.")

Aucune table n'a été créée dans la base de données SQLite.


In [9]:
customers = pd.read_csv('./customers.csv')
customers.head()
stores = pd.read_csv('./stores.csv')
stores.head()
transactions = pd.read_csv('./transactions.csv')
transactions.head()
products = pd.read_csv('./products.csv')
products.head()

,product_id,product_name,category,price
0,1,Product 1,Books,14.02
1,2,Product 2,Books,291.83
2,3,Product 3,Books,295.06
3,4,Product 4,Electronics,228.15
4,5,Product 5,Furniture,419.20


In [10]:
conn = sqlite3.connect(':memory:')
products.to_sql('products', conn, index=False, if_exists='replace')
customers.to_sql('customers', conn, index=False, if_exists='replace')
stores.to_sql('stores', conn, index=False, if_exists='replace')
transactions.to_sql('transactions', conn, index=False, if_exists='replace')
query = """
        SELECT *
        FROM transactions;
    """

result = pd.read_sql_query(query, conn)
result

,transaction_id,customer_id,product_id,store_id,date,amount
0,1,35,8,12,2023-01-01,379.09
1,2,236,34,16,2023-01-02,18.48
2,3,334,50,20,2023-01-03,783.51
3,4,325,46,20,2023-01-04,333.82
4,5,332,89,6,2023-01-05,329.74
...,...,...,...,...,...,...
995,996,271,62,8,2025-09-22,268.09
996,997,380,9,4,2025-09-23,829.86
997,998,46,22,14,2025-09-24,603.12
998,999,2,85,8,2025-09-25,882.43


### Exploration des Tables et Pré-Analyse

## 1 - Lister les noms des colonnes et le type de données pour chaque table.


In [11]:
query = """
        PRAGMA table_info(transactions)
    """



result = pd.read_sql_query(query, conn)
result


,cid,name,type,notnull,dflt_value,pk
0,0,transaction_id,INTEGER,0,None,0
1,1,customer_id,INTEGER,0,None,0
2,2,product_id,INTEGER,0,None,0
3,3,store_id,INTEGER,0,None,0
4,4,date,TEXT,0,None,0
5,5,amount,REAL,0,None,0


In [12]:
query = """
        PRAGMA table_info(stores)
    """



result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,store_id,INTEGER,0,None,0
1,1,store_name,TEXT,0,None,0
2,2,location,TEXT,0,None,0


In [13]:
query = """
        PRAGMA table_info(products)
    """



result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,product_id,INTEGER,0,None,0
1,1,product_name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,price,REAL,0,None,0


In [14]:
query = """
        PRAGMA table_info(customers)
    """



result = pd.read_sql_query(query, conn)
result

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,INTEGER,0,None,0
1,1,customer_name,TEXT,0,None,0
2,2,age,INTEGER,0,None,0
3,3,gender,TEXT,0,None,0


## 2 - Vérifier s'il y a des doublons dans la table transactions sur la colonne transaction_id.


In [15]:
query = """
        SELECT transaction_id, COUNT(*)
FROM transactions
GROUP BY transaction_id
HAVING COUNT(*) > 1;
    """



result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [transaction_id, COUNT(*)]
Index: []


## 3 - Identifier les clés primaires et les clés étrangères de chaque table.




In [16]:
query = """
SELECT *
FROM pragma_table_info('transactions')
WHERE pk > 0;
"""


result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


In [17]:
query = """
SELECT *
FROM pragma_table_info('stores')
WHERE pk > 0;
"""


result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


In [18]:
query = """
SELECT *
FROM pragma_table_info('products')
WHERE pk > 0;
"""


result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


In [19]:
query = """
SELECT *
FROM pragma_table_info('customers')
WHERE pk > 0;
"""


result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [cid, name, type, notnull, dflt_value, pk]
Index: []


In [20]:
query = """
SELECT *
FROM pragma_table_info('transactions')
WHERE pk = 0;
"""


result = pd.read_sql_query(query, conn)
print(result)

   cid            name     type  notnull dflt_value  pk
0    0  transaction_id  INTEGER        0       None   0
1    1     customer_id  INTEGER        0       None   0
2    2      product_id  INTEGER        0       None   0
3    3        store_id  INTEGER        0       None   0
4    4            date     TEXT        0       None   0
5    5          amount     REAL        0       None   0


## 4 - Vérifier si la colonne date dans la table transactions est stockée dans un format date, sinon, la convertir.


## 5 - Vérifier s'il existe des valeurs nulles dans les colonnes customer_id, product_id, store_id des transactions.




In [21]:
query = """
SELECT
    SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS customer_id_null_count,
    SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS product_id_null_count,
    SUM(CASE WHEN store_id IS NULL THEN 1 ELSE 0 END) AS store_id_null_count
FROM
    transactions;

"""


result = pd.read_sql_query(query, conn)
print(result)

   customer_id_null_count  product_id_null_count  store_id_null_count
0                       0                      0                    0


# Requêtes de Base



## 6- Lister toutes les transactions effectuées dans le magasin New York.



In [22]:
query = """
SELECT *
FROM transactions
JOIN stores ON transactions.store_id = stores.store_id
WHERE stores.location = 'New York';
"""
result = pd.read_sql_query(query, conn)
print(result)

     transaction_id  customer_id  product_id  store_id        date  amount  \
0                 7          146          78         1  2023-01-07   48.99   
1                12          500          81         1  2023-01-12   11.31   
2                29           70          52         1  2023-01-29  290.12   
3                51          400          62         1  2023-02-20  733.48   
4                76          452          29         1  2023-03-17  277.07   
..              ...          ...         ...       ...         ...     ...   
396             937          104          88        20  2025-07-25  905.52   
397             941          370          74        20  2025-07-29  308.36   
398             960          497           2        20  2025-08-17  127.38   
399             978          292          95        20  2025-09-04  218.93   
400             990          399          58        20  2025-09-16  514.18   

     store_id store_name  location  
0           1    Store 1  

## 7- Récupérer les noms des produits achetés par le customer_id 35.



In [23]:
query = """
SELECT product_name
FROM transactions
JOIN products ON transactions.product_id = products.product_id
WHERE transactions.customer_id = 35
;
"""
result = pd.read_sql_query(query, conn)
print(result)

  product_name
0    Product 8
1   Product 96


## 8- Lister les différents types de catégories de produits disponibles.



In [24]:
query = """
SELECT DISTINCT category
FROM products;
"""
result = pd.read_sql_query(query, conn)
print(result)

      category
0        Books
1  Electronics
2    Furniture
3     Clothing


## 9- Trouver le montant total des transactions pour le store_id 12.



In [25]:
query = """
SELECT SUM(amount) AS total_amount
FROM transactions
WHERE store_id = 12;
"""
result = pd.read_sql_query(query, conn)
print(result)

   total_amount
0      26796.29


## 10- Lister tous les clients de sexe féminin.



In [26]:
query = """
SELECT *
FROM customers
WHERE gender = "Female";
"""
result = pd.read_sql_query(query, conn)
print(result)

     customer_id customer_name  age  gender
0              2    Customer 2   40  Female
1              7    Customer 7   55  Female
2              9    Customer 9   51  Female
3             10   Customer 10   22  Female
4             11   Customer 11   69  Female
..           ...           ...  ...     ...
226          490  Customer 490   53  Female
227          494  Customer 494   35  Female
228          496  Customer 496   59  Female
229          497  Customer 497   50  Female
230          500  Customer 500   69  Female

[231 rows x 4 columns]


# Fonctions d’Agrégation et GROUP BY




## 11- Calculer la moyenne des montants des transactions par magasin.



In [27]:
query = """
SELECT store_id, AVG(amount) AS average_amount
FROM transactions
GROUP BY store_id
;
"""
result = pd.read_sql_query(query, conn)
print(result)

    store_id  average_amount
0          1      532.130000
1          2      496.529592
2          3      481.485789
3          4      563.986364
4          5      614.618200
5          6      465.471190
6          7      503.759412
7          8      461.190196
8          9      465.133200
9         10      504.817115
10        11      453.278361
11        12      546.863061
12        13      569.012857
13        14      479.562745
14        15      478.972679
15        16      493.564167
16        17      546.908519
17        18      602.175526
18        19      570.356531
19        20      496.091786


## 12- Trouver le magasin qui a réalisé le chiffre d'affaires le plus élevé.



In [28]:
query = """
SELECT store_id, SUM(amount) AS big_amount 
FROM transactions
ORDER BY store_id LIMIT 1
;
"""
result = pd.read_sql_query(query, conn)
print(result)

   store_id  big_amount
0        12    513915.4


## 13- Calculer le montant total dépensé par chaque client.



In [29]:
query = """
SELECT *
FROM transactions
;
"""
result = pd.read_sql_query(query, conn)
print(result)

     transaction_id  customer_id  product_id  store_id        date  amount
0                 1           35           8        12  2023-01-01  379.09
1                 2          236          34        16  2023-01-02   18.48
2                 3          334          50        20  2023-01-03  783.51
3                 4          325          46        20  2023-01-04  333.82
4                 5          332          89         6  2023-01-05  329.74
..              ...          ...         ...       ...         ...     ...
995             996          271          62         8  2025-09-22  268.09
996             997          380           9         4  2025-09-23  829.86
997             998           46          22        14  2025-09-24  603.12
998             999            2          85         8  2025-09-25  882.43
999            1000          253           4         8  2025-09-26  365.28

[1000 rows x 6 columns]


In [30]:
query = """
SELECT customer_id, SUM(amount) AS amount_per_customer
FROM transactions
GROUP BY customer_id
;
"""
result = pd.read_sql_query(query, conn)
print(result)

     customer_id  amount_per_customer
0              1              2528.13
1              2              2108.46
2              3               427.58
3              4              1447.50
4              5               574.77
..           ...                  ...
434          496              2437.91
435          497              1821.58
436          498              1672.92
437          499               486.39
438          500               384.58

[439 rows x 2 columns]


## 14- Trouver le produit le plus vendu en termes de quantité.



In [31]:
query = """
SELECT product_id, COUNT(*) AS total_sales_by_product
FROM transactions
GROUP BY product_id
ORDER BY total_sales_by_product DESC LIMIT 1;

"""
result = pd.read_sql_query(query, conn)
print(result)

   product_id  total_sales_by_product
0          45                      20


## 15- Lister les catégories de produits et le montant total des ventes par catégorie.



In [32]:
query = """
SELECT p.category, SUM(t.amount) AS total_sales
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.category
ORDER BY total_sales DESC;

"""
result = pd.read_sql_query(query, conn)
print(result)

      category  total_sales
0        Books    151828.24
1    Furniture    127514.47
2  Electronics    125958.34
3     Clothing    108614.35


# Clauses HAVING vs WHERE




## 16- Lister les clients qui ont effectué plus de 5 transactions.



In [33]:
query = """
SELECT customer_id, COUNT(*) AS transaction_count
FROM transactions
GROUP BY customer_id
HAVING COUNT(*) > 5;

"""
result = pd.read_sql_query(query, conn)
print(result)

   customer_id  transaction_count
0           61                  6
1          141                  6
2          277                  6


## 17- Trouver les produits qui ont généré plus de 1000 en ventes totales.



In [34]:
query = """
SELECT p.product_id, SUM(t.amount) AS total_sales
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.product_id
HAVING total_sales > 1000;


"""
result = pd.read_sql_query(query, conn)
print(result)

    product_id  total_sales
0            1      4334.66
1            2      5735.32
2            3      5909.48
3            4      3590.36
4            5      4182.23
..         ...          ...
95          96      6980.49
96          97      6947.89
97          98      4940.70
98          99      5780.27
99         100      2560.02

[100 rows x 2 columns]


## 18- Lister les magasins ayant une moyenne de transaction supérieure à 200.



In [35]:
query = """
SELECT p.product_id, SUM(t.amount) AS total_sales
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.product_id
HAVING SUM(t.amount) > 1000;


"""
result = pd.read_sql_query(query, conn)
print(result)

    product_id  total_sales
0            1      4334.66
1            2      5735.32
2            3      5909.48
3            4      3590.36
4            5      4182.23
..         ...          ...
95          96      6980.49
96          97      6947.89
97          98      4940.70
98          99      5780.27
99         100      2560.02

[100 rows x 2 columns]


## 19- Récupérer les produits appartenant à la catégorie Books et ayant généré plus de 500 EN AMOUNT en ventes.

In [36]:
query = """
SELECT p.category, p.product_id, SUM(t.amount) AS total_sales
FROM transactions t
JOIN products p ON t.product_id = p.product_id
WHERE p.category = "Books"
GROUP BY p.product_id
HAVING total_sales > 500;

"""
result = pd.read_sql_query(query, conn)
print(result)

   category  product_id  total_sales
0     Books           1      4334.66
1     Books           2      5735.32
2     Books           3      5909.48
3     Books           6      3147.49
4     Books           8      4192.44
5     Books           9      7749.04
6     Books          14      7558.51
7     Books          20      5097.30
8     Books          31      3538.25
9     Books          33      6547.09
10    Books          39      7603.02
11    Books          42      7611.42
12    Books          47      2999.79
13    Books          50      4482.85
14    Books          52      5492.80
15    Books          54      7047.71
16    Books          55      4650.90
17    Books          56      1982.06
18    Books          61      7822.99
19    Books          62      6269.32
20    Books          65      3846.50
21    Books          69      4718.06
22    Books          72      5466.35
23    Books          73      3885.76
24    Books          79      8666.74
25    Books          86      4495.20
2

## 20- Trouver les clients ayant dépensé plus de 1000 au total.



In [37]:
query = """
SELECT c.customer_id, SUM(t.amount) AS total_sales
FROM transactions t
JOIN customers c ON t.customer_id = c.customer_id
GROUP BY t.customer_id
HAVING total_sales > 1000
;

"""
result = pd.read_sql_query(query, conn)
print(result)

     customer_id  total_sales
0              1      2528.13
1              2      2108.46
2              4      1447.50
3              6      1934.12
4              7      1303.84
..           ...          ...
217          494      2268.27
218          495      1814.12
219          496      2437.91
220          497      1821.58
221          498      1672.92

[222 rows x 2 columns]


# Fonctions de Manipulation de Chaînes et Dates




## 21- Extraire l’année des dates de transactions et compter le nombre de transactions par année.

In [38]:
query = """
SELECT DISTINCT  strftime('%Y', date) AS year, COUNT(transaction_id)
FROM transactions
GROUP BY year
ORDER BY year

;

"""
result = pd.read_sql_query(query, conn)
print(result)

   year  COUNT(transaction_id)
0  2023                    365
1  2024                    366
2  2025                    269


## 22- Concaténer le nom du client avec son âge.



In [39]:
query = """
SELECT customer_name || ' (Age: ' || age || ')' 
FROM customers;

"""
result = pd.read_sql_query(query, conn)
print(result)

    customer_name || ' (Age: ' || age || ')'
0                       Customer 1 (Age: 25)
1                       Customer 2 (Age: 40)
2                       Customer 3 (Age: 30)
3                       Customer 4 (Age: 33)
4                       Customer 5 (Age: 43)
..                                       ...
495                   Customer 496 (Age: 59)
496                   Customer 497 (Age: 50)
497                   Customer 498 (Age: 51)
498                   Customer 499 (Age: 35)
499                   Customer 500 (Age: 69)

[500 rows x 1 columns]


## 23- Remplacer tous les espaces par des underscores dans les noms des produits.



In [40]:
query = """
SELECT customer_name || ' (Age: ' || age || ')' 
FROM customers;

"""
result = pd.read_sql_query(query, conn)
print(result)

    customer_name || ' (Age: ' || age || ')'
0                       Customer 1 (Age: 25)
1                       Customer 2 (Age: 40)
2                       Customer 3 (Age: 30)
3                       Customer 4 (Age: 33)
4                       Customer 5 (Age: 43)
..                                       ...
495                   Customer 496 (Age: 59)
496                   Customer 497 (Age: 50)
497                   Customer 498 (Age: 51)
498                   Customer 499 (Age: 35)
499                   Customer 500 (Age: 69)

[500 rows x 1 columns]


## 24- Transformer en majuscules les noms des magasins.



In [41]:
query = """
SELECT UPPER(store_name) AS store_name
FROM stores;

"""
result = pd.read_sql_query(query, conn)
print(result)

   store_name
0     STORE 1
1     STORE 2
2     STORE 3
3     STORE 4
4     STORE 5
5     STORE 6
6     STORE 7
7     STORE 8
8     STORE 9
9    STORE 10
10   STORE 11
11   STORE 12
12   STORE 13
13   STORE 14
14   STORE 15
15   STORE 16
16   STORE 17
17   STORE 18
18   STORE 19
19   STORE 20


# Joins


## 25- Joindre la table transactions avec stores et afficher le nom du magasin pour chaque transaction.

In [42]:
query = """
SELECT *
FROM stores

;

"""
result = pd.read_sql_query(query, conn)
print(result)

    store_id store_name     location
0          1    Store 1     New York
1          2    Store 2  Los Angeles
2          3    Store 3     New York
3          4    Store 4     New York
4          5    Store 5      Phoenix
5          6    Store 6  Los Angeles
6          7    Store 7  Los Angeles
7          8    Store 8      Houston
8          9    Store 9     New York
9         10   Store 10     New York
10        11   Store 11  Los Angeles
11        12   Store 12  Los Angeles
12        13   Store 13  Los Angeles
13        14   Store 14     New York
14        15   Store 15      Houston
15        16   Store 16     New York
16        17   Store 17      Phoenix
17        18   Store 18      Chicago
18        19   Store 19      Phoenix
19        20   Store 20     New York


In [43]:
query = """
SELECT s.store_name AS store_name, t.transaction_id
FROM transactions t
JOIN stores s ON s.store_id = t.store_id
GROUP BY store_name
;

"""
result = pd.read_sql_query(query, conn)
print(result)

   store_name  transaction_id
0     Store 1               7
1    Store 10              58
2    Store 11              20
3    Store 12               1
4    Store 13              10
5    Store 14              24
6    Store 15               8
7    Store 16               2
8    Store 17              16
9    Store 18              83
10   Store 19              19
11    Store 2              28
12   Store 20               3
13    Store 3              23
14    Store 4               6
15    Store 5              35
16    Store 6               5
17    Store 7              45
18    Store 8              13
19    Store 9              30


## 26- Joindre les tables transactions et products pour obtenir le nom du produit acheté dans chaque transaction.

In [44]:
query = """
SELECT p.product_name AS product_name, t.transaction_id
FROM transactions t
JOIN products p ON p.product_id = t.product_id
GROUP BY product_name
;

"""
result = pd.read_sql_query(query, conn)
print(result)

   product_name  transaction_id
0     Product 1              20
1    Product 10              10
2   Product 100              24
3    Product 11             140
4    Product 12              68
..          ...             ...
95   Product 95               8
96   Product 96             212
97   Product 97              16
98   Product 98             162
99   Product 99              52

[100 rows x 2 columns]


## 27- Faire une jointure entre transactions, products, et stores pour afficher le produit acheté, le magasin et le montant.

In [45]:
query = """
SELECT p.product_name AS product_name, s.store_id, t.transaction_id
FROM transactions t
JOIN products p ON p.product_id = t.product_id
JOIN stores s ON s.store_id = t.store_id
GROUP BY product_name
;

"""
result = pd.read_sql_query(query, conn)
print(result)

   product_name  store_id  transaction_id
0     Product 1        11              20
1    Product 10        13              10
2   Product 100        14              24
3    Product 11         2             140
4    Product 12         5              68
..          ...       ...             ...
95   Product 95        15               8
96   Product 96        13             212
97   Product 97        17              16
98   Product 98         3             162
99   Product 99        12              52

[100 rows x 3 columns]


## 28- Lister les transactions pour les clients de plus de 30 ans.



In [46]:
query = """
SELECT * FROM customers

"""
result = pd.read_sql_query(query, conn)
print(result)

     customer_id customer_name  age  gender
0              1    Customer 1   25    Male
1              2    Customer 2   40  Female
2              3    Customer 3   30    Male
3              4    Customer 4   33    Male
4              5    Customer 5   43    Male
..           ...           ...  ...     ...
495          496  Customer 496   59  Female
496          497  Customer 497   50  Female
497          498  Customer 498   51    Male
498          499  Customer 499   35    Male
499          500  Customer 500   69  Female

[500 rows x 4 columns]


In [47]:
query = """
SELECT t.transaction_id, c.customer_name AS customer_name, c.age AS les_vieux_de_plus_de_30_ans
FROM transactions t
JOIN customers c ON c.customer_id = t.customer_id
WHERE c.age > 30
GROUP BY customer_name
;

"""
result = pd.read_sql_query(query, conn)
print(result)

     transaction_id customer_name  les_vieux_de_plus_de_30_ans
0               303  Customer 101                           33
1               101  Customer 102                           67
2               326  Customer 105                           39
3               583  Customer 106                           36
4               157  Customer 107                           68
..              ...           ...                          ...
329             495   Customer 92                           41
330             139   Customer 95                           57
331             206   Customer 96                           44
332             376   Customer 98                           58
333              57   Customer 99                           61

[334 rows x 3 columns]


## 29- Lister tous les produits achetés par les clients de sexe masculin.



In [48]:
query = """
SELECT t.transaction_id, c.gender AS male_alpha
FROM transactions t
JOIN customers c ON c.customer_id = t.customer_id
WHERE c.gender = 'Male'
GROUP BY t.transaction_id
;

"""
result = pd.read_sql_query(query, conn)
print(result)

     transaction_id male_alpha
0                 2       Male
1                 3       Male
2                 4       Male
3                 7       Male
4                11       Male
..              ...        ...
545             988       Male
546             989       Male
547             991       Male
548             993       Male
549             995       Male

[550 rows x 2 columns]


# Sous-requêtes (Subqueries) et CTE




## 30- Trouver les clients qui ont effectué une transaction pour un montant supérieur à la moyenne des transactions.

In [49]:
query = """
SELECT 
    c.customer_id,
    c.customer_name,
    t.amount
FROM 
    transactions t
JOIN 
    customers c ON c.customer_id = t.customer_id
WHERE 
    t.amount > (SELECT AVG(amount) FROM transactions);
 

"""
result = pd.read_sql_query(query, conn)
print(result)

     customer_id customer_name  amount
0            334  Customer 334  783.51
1            379  Customer 379  811.79
2             42   Customer 42  515.03
3             28   Customer 28  826.28
4            453  Customer 453  888.98
..           ...           ...     ...
497          351  Customer 351  934.32
498          189  Customer 189  724.74
499          380  Customer 380  829.86
500           46   Customer 46  603.12
501            2    Customer 2  882.43

[502 rows x 3 columns]


## 31- Lister les produits ayant un prix supérieur au prix moyen de leur catégorie.



In [54]:
query = """
SELECT 
    p.product_name,
    p.category,
    p.price
FROM 
    products p
WHERE 
    p.price > (
    SELECT AVG(p2.price) FROM products p2
    WHERE p2.category = p.category --Relie chaque produit à sa catégorie
    )
 ;

"""
result = pd.read_sql_query(query, conn)
print(result)

   product_name     category   price
0     Product 2        Books  291.83
1     Product 3        Books  295.06
2     Product 5    Furniture  419.20
3     Product 8        Books  388.96
4     Product 9        Books  419.09
5    Product 10  Electronics  332.69
6    Product 11     Clothing  267.75
7    Product 12     Clothing  450.63
8    Product 14        Books  309.15
9    Product 15  Electronics  480.60
10   Product 17    Furniture  252.33
11   Product 18  Electronics  273.78
12   Product 19    Furniture  388.56
13   Product 20        Books  400.85
14   Product 21    Furniture  229.55
15   Product 25     Clothing  498.78
16   Product 27     Clothing  404.44
17   Product 28  Electronics  488.36
18   Product 29    Furniture  430.69
19   Product 30  Electronics  369.34
20   Product 31        Books  488.03
21   Product 32    Furniture  343.67
22   Product 34  Electronics  392.02
23   Product 35  Electronics  451.66
24   Product 40     Clothing  378.11
25   Product 42        Books  361.28
2

## 32- Utiliser un CTE pour calculer le montant total des transactions par client, puis filtrer pour trouver ceux ayant dépensé plus de 500.

In [56]:
query = """
WITH total_amount_per_client AS (
SELECT c.customer_name,  SUM(t.amount) as total_spent
FROM transactions t
JOIN customers c ON t.customer_id = c.customer_id
GROUP BY c.customer_name
HAVING total_spent > 500

)

SELECT customer_name, total_spent
FROM total_amount_per_client
 ;

"""
result = pd.read_sql_query(query, conn)
print(result)

    customer_name  total_spent
0      Customer 1      2528.13
1     Customer 10      1854.36
2    Customer 101      2295.63
3    Customer 102       682.86
4    Customer 104      1275.28
..            ...          ...
338   Customer 94       842.69
339   Customer 95      2541.37
340   Customer 96       565.53
341   Customer 98      1101.91
342   Customer 99      1943.84

[343 rows x 2 columns]


## 33- Lister les produits qui ont été achetés plus de 10 fois et appartenant à la catégorie Electronics.

# Fonctions Fenêtre (Window Functions)




## 34- Attribuer un rang aux transactions en fonction du montant dans chaque magasin.



## 35- Calculer la somme cumulée du montant des transactions pour chaque client.



## 36- Attribuer un numéro de rang aux magasins basé sur le chiffre d'affaires total.



## 37- Calculer le pourcentage de contribution de chaque transaction au total des ventes du magasin.